<a href="https://colab.research.google.com/github/mynameis-sohee/Recommendation-Algorithm/blob/main/Tutorial/Contents-Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import io
import pandas as pd
import numpy as np

In [2]:
uploaded = files.upload()

Saving tmdb_5000_credits.csv to tmdb_5000_credits.csv
Saving tmdb_5000_movies.csv to tmdb_5000_movies.csv


In [4]:
df_credits = pd.read_csv(io.BytesIO(uploaded['tmdb_5000_credits.csv']))
df_movies = pd.read_csv(io.BytesIO(uploaded['tmdb_5000_movies.csv']))

In [6]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
# 텍스트 자동 파싱
from ast import literal_eval

# 육안으로는 큰 차이는 없어 보이나, 데이터가 객체로 들어가 있도록 바뀌는 것임
df_movies['genres']=df_movies['genres'].apply(literal_eval)

In [15]:
# name 리스트화
df_movies['genres']=df_movies['genres'].apply(lambda x: [y['name'] for y in x])

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 적용 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
df_movies['genres_literal']=df_movies['genres'].apply(lambda x: (' ').join(x))

In [22]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(df_movies['genres_literal'])
genre_mat.shape

(4803, 276)

In [25]:
# 장르에 따른 영화별 코사인 유사도 추출
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
# 2차원 배열 형태
print(genre_sim[:2])

# 반환값 numpy & 실제 데이터 추출은 df에 해야 할 경우 argsort(내림차순) 을 이용
# ex: 0번 영화에 대해 코사인 유사도로 가장 유사한 순으로 갖고 있는 다른 영화들의 인덱스를 갖고 있음
genre_sim_sorted_ind=genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]
[[   0 3494  813 ... 3038 3037 2401]]


In [27]:
# 특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index, :(top_n)]

  print(similar_indexes)
  # index를 1차원 변환
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]


In [31]:
# 비슷한 장르 추출
similar_movies = find_sim_movie(df_movies, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1
